# Minimizing the number of trucks to pack pallets
 - Correction
A logistic firm has to load $m$ trucks $\{1, \dots, m\}$ in order to daily serve a set of costumers. In the trucks, $n$ pallets $\{1, \dots , n\}$ must be loaded, the $j^{th}$ of them is characterized by a weight $w_j$ . The $i^{th}$ truck has a weight capacity of $b_i$ . The problem asks for determining
the minimum number of trucks to pack all the pallets.

## Question 1

Write a ILP model to minimize the total number of trucks necessary to serve all the clients. Identify the decisions that must be taken and the corresponding decision variables that have to be used. Identify and comment the objective function of the problem and the constraints.


**CORRECTION :**


Let $I = \{1, \dots, m\}$ and $J = \{1, \dots, n\}$

For $i \in I$, let $y_i$ be equal to 1 if truck number $i$ is used, and 0 otherwise.

For $i\in I$ and $j\in J$, let $x_{ij}$ be equal to 1 if pallet number $j$ is carried by vehicle number $i$.

The problem can be formulated as follows.

\begin{align}
&\min \sum_{i \in I} y_i\\
&\sum_{i \in I} x_{ij} = 1 & \quad \forall j \in J\\
&\sum_{j \in J} w_j~x_{ij} \le b_i~y_i & \quad \forall i \in I\\
&y_i \in \{0,1\} & \forall i \in I\\
&x_{ij} \in \{0,1\} & \forall i \in I, j \in J
\end{align}


## Question 2

Consider the following case in which there are 10 pallets ($n = 10$) and 10 available trucks ($m = 10$), the capacity of each truck is $100$. The weights of the pallets are:

 &nbsp;    | $j=1$ | $j=2$ | $j=3$ | $j=4$ | $j=5$ | $j=6$ | $j=7$ | $j=8$ | $j=9$ | $j=10$
---|---|----|---|---|---|---|---|---|---|---|
$w_1$ | 52 | 51 | 28 | 27 | 26| 25 | 23| 23| 23| 22

Define a heuristic to find a good solution. How many trucks do we need?

**CORRECTION :**


We can order the pallets in decreasing weights.
For each pallet in that order, put it in the truck having the minimum available capacity greater than the pallet weight, or use a new truck to pack it if there no such truck.

When we apply this heuristic for the instance, we get:

- Truck 1: 52, 28
- Truck 2: 51, 27, 22
- Truck 3: 26, 25, 23, 23
- Truck 4: 23

We then need at most 4 trucks to pack all pallets.

## Question 3

Implement the MILP to solve to optimality the instance of Question 2 with $m$ equal to the number of trucks used in the heuristic.

In [ ]:
%pip install mip

In [ ]:
pallet_weights = [ 52, 51, 28, 27, 26, 25, 23, 23, 23, 22]

In [ ]:
from mip import *

In [ ]:
##################
#   Correction   #
##################


def solve_packing(truck_capacity, weights, number_of_trucks):
    m = Model()
    I = range(number_of_trucks)
    J = range(len(weights))
    x = [
            [m.add_var(var_type = BINARY) for j in J] for i in I
    ]
    y = [ m.add_var(var_type = BINARY) for i in I]
    m.objective = minimize(xsum(y[i] for i in I))

    # for each pallet
    for j in J:
        m += xsum(x[i][j] for i in I) == 1

    # for each truck
    for i in I:
        m += xsum( weights[j] * x[i][j] for j in J) <= truck_capacity * y[i]
    m.verbose = 0 # to remove the output of the solution process
    status = m.optimize()
    print(f"Number of necessary trucks: {m.objective_value}")
    for i in I:
        if y[i].x >= 0.99:
            pallets = [j+1 for j in J if x[i][j].x >= 0.99]
            print(f"pallets inside truck {i}: {pallets}")

solve_packing(100, pallet_weights, 4)      